# HOUSING PRICE

In [15]:
import pandas as pd
import matplotlib.pyplot as plt


In [16]:
df_hp=pd.read_csv("/content/Bengaluru_House_Data.csv")
df_hp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13320 entries, 0 to 13319
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     13320 non-null  object 
 1   availability  13320 non-null  object 
 2   location      13319 non-null  object 
 3   size          13304 non-null  object 
 4   society       7818 non-null   object 
 5   total_sqft    13320 non-null  object 
 6   bath          13247 non-null  float64
 7   balcony       12711 non-null  float64
 8   price         13320 non-null  float64
dtypes: float64(3), object(6)
memory usage: 936.7+ KB


In [17]:
def rep_size(obj):
  obj=obj.split(" ")
  obj=obj[0]
  return obj


In [18]:
col=["society"]
df_hp.drop(columns=col,inplace=True)

In [19]:
df_hp = df_hp.dropna()


In [20]:
def rep_avai(obj):
  obj=obj.split("-")
  if(len(obj)>1):
    #print(obj)
    obj=obj[1]
    return obj
  return obj[0]

In [21]:
df_hp["size"]=df_hp["size"].apply(rep_size)

In [22]:
df_hp["availability"]=df_hp["availability"].apply(rep_avai)

In [23]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder(categories=[['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep','Oct', 'Nov', 'Dec', 'Ready To Move']])
df_hp['availability'] = ordinal_encoder.fit_transform(df_hp[['availability']])
"""
from sklearn.preprocessing import OrdinalEncoder

# Define your ordinal categories
categories = [['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'Ready To Move']]

# Initialize OrdinalEncoder with your categories
ordinal_encoder = OrdinalEncoder(categories=categories)

# Fit and transform your ordinal column
encoded_values = ordinal_encoder.fit_transform(df_hp[['availability']])

# Add 1 to the encoded values to start from 1
encoded_values += 1

# Assign the encoded values to a new column
df_hp['availability_encoded'] = encoded_values

# Access the learned categories along with their numerical representations
category_mapping = dict(zip(ordinal_encoder.categories_[0], range(1, len(ordinal_encoder.categories_[0]) + 1)))

print("Category mapping:")
print(category_mapping)


#df_hp["availability"]=df_hp["availability"].replace('Jan',0)"""

'\nfrom sklearn.preprocessing import OrdinalEncoder\n\n# Define your ordinal categories\ncategories = [[\'Jan\', \'Feb\', \'Mar\', \'Apr\', \'May\', \'Jun\', \'Jul\', \'Aug\', \'Sep\', \'Oct\', \'Nov\', \'Dec\', \'Ready To Move\']]\n\n# Initialize OrdinalEncoder with your categories\nordinal_encoder = OrdinalEncoder(categories=categories)\n\n# Fit and transform your ordinal column\nencoded_values = ordinal_encoder.fit_transform(df_hp[[\'availability\']])\n\n# Add 1 to the encoded values to start from 1\nencoded_values += 1\n\n# Assign the encoded values to a new column\ndf_hp[\'availability_encoded\'] = encoded_values\n\n# Access the learned categories along with their numerical representations\ncategory_mapping = dict(zip(ordinal_encoder.categories_[0], range(1, len(ordinal_encoder.categories_[0]) + 1)))\n\nprint("Category mapping:")\nprint(category_mapping)\n\n\n#df_hp["availability"]=df_hp["availability"].replace(\'Jan\',0)'

In [24]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the 'availability' column
df_hp['area_type'] = label_encoder.fit_transform(df_hp['area_type'])

In [25]:
df_hp['location'] = label_encoder.fit_transform(df_hp['location'])
#df_hp = pd.get_dummies(df_hp, columns=['location'])


In [26]:
import re
def total_sq_modify(obj):
    if(re.search(r'\D+', obj)):
        output = re.search(r'\D+', obj).group()
        if (output=="Acres"):
          numeric_part = re.search(r'\d+', obj).group()
          obj=float(numeric_part)*4046.86
        elif (output=="Sq. Yards"):
          numeric_part = re.search(r'\d+', obj).group()
          obj=float(numeric_part)*0.836127
        elif (output=="Sq. Meter"):
          numeric_part = re.search(r'\d+', obj).group()
          obj=float(numeric_part)*1
        elif (output=="Cents"):
          numeric_part = re.search(r'\d+', obj).group()
          obj=float(numeric_part)*40.47
        elif (output=="Guntha"):
          numeric_part = re.search(r'\d+', obj).group()
          obj=float(numeric_part)*101.171367
        elif (output=="Grounds"):
          numeric_part = re.search(r'\d+', obj).group()
          obj=float(numeric_part)*222.9675
        else:
            numeric_part = re.search(r'\d+', obj).group()
            obj=float(numeric_part)
    else:
        numeric_part = re.search(r'\d+', obj).group()
        obj=float(numeric_part)


    return obj

In [27]:
df_hp['total_sqft']=df_hp['total_sqft'].apply(total_sq_modify)

In [34]:
# Extract values from the 'total_sqft' column
unique_total_sqft = df_hp['total_sqft']

# Convert the unique values to a DataFrame
unique_total_sqft_df = pd.DataFrame(unique_total_sqft)

# Write the DataFrame to a text file
unique_total_sqft_df.to_csv('unique_total_sqft.txt', header=False, index=False)


In [40]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

target_column = 'price'
X = df_hp.drop(columns=[target_column])
y = df_hp[target_column]
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=42) # 0.25 * 0.8 = 0.2

# Step 3: Create and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Step 4: Evaluate the model on the validation set
y_val_pred = model.predict(X_val)

# Calculate Mean Squared Error (MSE) on the validation set
val_mse = mean_squared_error(y_val, y_val_pred)
print("Validation Mean Squared Error:", val_mse)
# Step 5: Evaluate the model on the testing set
y_test_pred = model.predict(X_test)

# Calculate Mean Squared Error (MSE) on the testing set
test_mse = mean_squared_error(y_test, y_test_pred)
print("Test Mean Squared Error:", test_mse)
import numpy as np

# Calculate Root Mean Squared Error (RMSE) for validation set
val_rmse = np.sqrt(val_mse)
print("Validation Root Mean Squared Error:", val_rmse)

# Calculate Root Mean Squared Error (RMSE) for test set
test_rmse = np.sqrt(test_mse)
print("Test Root Mean Squared Error:", test_rmse)



Validation Mean Squared Error: 18148.444603398508
Test Mean Squared Error: 13316.371269824758
Validation Root Mean Squared Error: 134.7161631111817
Test Root Mean Squared Error: 115.39658257428925
